In [ ]:
# 구글 드라이브에서 크롤링 한 파일 가져와서 라벨링 및 임베딩
import os
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build
from sentence_transformers import SentenceTransformer
import chromadb
import openai
from datetime import datetime
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
openai.api_key = api_key

# Google Drive API 인증 설정
SERVICE_ACCOUNT_FILE = 'service_account.json'  # JSON 파일 경로
SCOPES = ['https://www.googleapis.com/auth/drive']

credentials = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
drive_service = build('drive', 'v3', credentials=credentials)

# 공유 드라이브의 폴더 ID 설정 (공유 드라이브 URL에서 추출)
shared_folder_id = "1GwbcEfpaCbcbkhBpXjR4GOO6UFVjceiz"  # 실제 공유 드라이브 ID로 대체

# SentenceTransformer 및 ChromaDB 초기화
model = SentenceTransformer('all-MiniLM-L6-v2')
client = chromadb.PersistentClient()

# 기존 컬렉션 삭제 후 생성
if "crawled_data" in [col.name for col in client.list_collections()]:
    client.delete_collection(name="crawled_data")
collection = client.create_collection(name="crawled_data")


# 라벨링 함수 수정
def label_topic(content):
    try:
        response = openai.chat.completions.create(
            messages=[
                {"role": "system", "content": "당신은 문서를 분석하여 적절한 주제를 할당하는 어시스턴트입니다. 가능한 주제는 '주거', '취업', '문화활동', '금융, 조례'입니다. 각 문서를 분석하여 관련이 높은 주제를 선택하세요. 하나만 선택하기 힘들다면 중복도 괜찮습니다."},
                {"role": "user", "content": f"다음 문서의 주제를 분석하고 적합한 주제를 단어로만 반환하세요:\n\n{content}"}
            ],
            model="gpt-4o",
        )
        label = response.choices[0].message.content
        print(f"주제 라벨링 결과: {label}")
        return label

    except Exception as e:
        print(f"라벨링 실패: {e}")
        return "기타"  # 실패 시 기본 라벨

# 지역 함수 수정
def label_region(content):
    try:
        response = openai.chat.completions.create(
            messages=[
                {"role": "system", "content": "당신은 문서를 분석하여 관련된 지역 정보를 파악하는 역할을 맡고 있습니다. 문서를 검토하여 해당 문서와 가장 관련이 있는 지역을 '시/군/구' 단위로 식별하세요. 결과는 '서울특별시 성동구'처럼 시와 구/군 이름이 포함된 전체 이름 형식으로 반환하세요. 만약 특정 지역과의 관련성이 없다면, '전국'이라는 값을 반환하세요."},
                {"role": "user", "content": f"다음 문서의 주제를 분석하고 적합한 지역을 단어로만 반환하세요:\n\n{content}"}
            ],
            model="gpt-4o",
        )
        region = response.choices[0].message.content
        print(f"지역 라벨링 결과: {region}")
        return region

    except Exception as e:
        print(f"지역 실패: {e}")
        return "기타"  # 실패 시 기본 라벨

# 모집 시작 날짜 라벨링 함수
def label_recruit_start_date(content):
    try:
        response = openai.chat.completions.create(
            messages=[
                {"role": "system", "content": (
                    "문서를 분석하여 모집 시작 날짜를 반환하세요. "
                    "결과는 반드시 아래 형식 중 하나를 따라야 합니다:\n\n"
                    "1. 'YYYY.MM.DD' 형식으로 날짜를 반환합니다. 예: '2024.01.15'\n"
                    "2. 모집 시작 날짜가 명시되지 않은 경우: '모집 시작 날짜 없음'\n"
                    "3. 문서에 모집과 관련된 내용이 없을 경우: '모집과 관련된 내용 없음'\n"
                    "4. 모집이 당일 시작된다는 경우: '오늘 모집 시작'\n\n"
                    "예시:\n"
                    "- 문서: '모집은 2024년 1월 15일부터 시작됩니다.'\n"
                    "  결과: '2024.01.15'\n"
                    "- 문서: '이 프로그램은 언제든지 지원 가능합니다.'\n"
                    "  결과: '모집 시작 날짜 없음'\n"
                    "- 문서: '이 프로그램은 오늘부터 시작됩니다.'\n"
                    "  결과: '오늘 모집 시작'\n"
                    "- 문서: '지원 절차는 없습니다.'\n"
                    "  결과: '모집과 관련된 내용 없음'\n"
                )},
                {"role": "user", "content": f"다음 문서를 분석하고 모집 시작 날짜를 반환하세요:\n\n{content}"}
            ],
            model="gpt-4",
        )
        recruit_start_date = response.choices[0].message.content.strip()
        print(f"모집 시작 날짜 라벨링 결과: {recruit_start_date}")
        return recruit_start_date
    except Exception as e:
        print(f"모집 시작 날짜 라벨링 실패: {e}")
        return "모집 시작 날짜 없음"


# 모집 마감 날짜 라벨링 함수
def label_recruit_end_date(content):
    try:
        response = openai.chat.completions.create(
            messages=[
                {"role": "system", "content": (
                    "문서를 분석하여 모집 마감 날짜를 반환하세요. "
                    "결과는 반드시 아래 형식 중 하나를 따라야 합니다:\n\n"
                    "1. 'YYYY.MM.DD' 형식으로 날짜를 반환합니다. 예: '2024.01.31'\n"
                    "2. 모집 마감 날짜가 명시되지 않은 경우: '모집 마감 날짜 없음'\n"
                    "3. 문서에 모집과 관련된 내용이 없을 경우: '모집과 관련된 내용 없음'\n"
                    "4. 모집이 계속 진행된다는 경우: '모집 종료 없음'\n\n"
                    "예시:\n"
                    "- 문서: '모집은 2024년 1월 31일까지 진행됩니다.'\n"
                    "  결과: '2024.01.31'\n"
                    "- 문서: '모집 기간은 상시 진행됩니다.'\n"
                    "  결과: '모집 종료 없음'\n"
                    "- 문서: '모집 일정은 추후 공지됩니다.'\n"
                    "  결과: '모집 마감 날짜 없음'\n"
                    "- 문서: '이 프로그램은 지원 절차가 없습니다.'\n"
                    "  결과: '모집과 관련된 내용 없음'\n"
                )},
                {"role": "user", "content": f"다음 문서를 분석하고 모집 마감 날짜를 반환하세요:\n\n{content}"}
            ],
            model="gpt-4",
        )
        recruit_end_date = response.choices[0].message.content.strip()
        print(f"모집 마감 날짜 라벨링 결과: {recruit_end_date}")
        return recruit_end_date
    except Exception as e:
        print(f"모집 마감 날짜 라벨링 실패: {e}")
        return "모집 마감 날짜 없음"

def label_implementation_start_date(content):
    try:
        response = openai.chat.completions.create(
            messages=[
                {"role": "system", "content": (
                    "문서를 분석하여 시행 시작 날짜를 반환하세요. "
                    "결과는 반드시 아래 형식 중 하나를 따라야 합니다:\n\n"
                    "1. 'YYYY.MM.DD' 형식으로 날짜를 반환합니다. 예: '2024.03.01'\n"
                    "2. 시행 시작 날짜가 명시되지 않은 경우: '시행 시작 날짜 없음'\n"
                    "3. 문서에 시행과 관련된 내용이 없을 경우: '시행과 관련된 내용 없음'\n"
                    "4. 시행이 오늘부터 시작된다는 경우: '오늘 시행 시작'\n\n"
                    "예시:\n"
                    "- 문서: '이 프로그램은 2024년 3월 1일부터 시행됩니다.'\n"
                    "  결과: '2024.03.01'\n"
                    "- 문서: '이 프로그램은 즉시 시작됩니다.'\n"
                    "  결과: '오늘 시행 시작'\n"
                    "- 문서: '시행 일정은 추후 결정됩니다.'\n"
                    "  결과: '시행 시작 날짜 없음'\n"
                    "- 문서: '시행 절차는 없습니다.'\n"
                    "  결과: '시행과 관련된 내용 없음'\n"
                )},
                {"role": "user", "content": f"다음 문서를 분석하고 시행 시작 날짜를 반환하세요:\n\n{content}"}
            ],
            model="gpt-4",
        )
        implementation_start_date = response.choices[0].message.content.strip()
        print(f"시행 시작 날짜 라벨링 결과: {implementation_start_date}")
        return implementation_start_date
    except Exception as e:
        print(f"시행 시작 날짜 라벨링 실패: {e}")
        return "시행 시작 날짜 없음"


# 시행 마감 날짜 라벨링 함수
def label_implementation_end_date(content):
    try:
        response = openai.chat.completions.create(
            messages=[
                {"role": "system", "content": (
                    "문서를 분석하여 시행 마감 날짜를 반환하세요. "
                    "결과는 반드시 아래 형식 중 하나를 따라야 합니다:\n\n"
                    "1. 'YYYY.MM.DD' 형식으로 날짜를 반환합니다. 예: '2024.12.31'\n"
                    "2. 시행 마감 날짜가 명시되지 않은 경우: '시행 마감 날짜 없음'\n"
                    "3. 문서에 시행과 관련된 내용이 없을 경우: '시행과 관련된 내용 없음'\n"
                    "4. 시행이 계속 진행된다는 경우: '시행 종료 없음'\n\n"
                    "예시:\n"
                    "- 문서: '이 프로그램은 2024년 12월 31일까지 시행됩니다.'\n"
                    "  결과: '2024.12.31'\n"
                    "- 문서: '이 프로그램은 계속적으로 시행됩니다.'\n"
                    "  결과: '시행 종료 없음'\n"
                    "- 문서: '시행 일정은 추후 공지됩니다.'\n"
                    "  결과: '시행 마감 날짜 없음'\n"
                    "- 문서: '시행 절차는 없습니다.'\n"
                    "  결과: '시행과 관련된 내용 없음'\n"
                )},
                {"role": "user", "content": f"다음 문서를 분석하고 시행 마감 날짜를 반환하세요:\n\n{content}"}
            ],
            model="gpt-4",
        )
        implementation_end_date = response.choices[0].message.content.strip()
        print(f"시행 마감 날짜 라벨링 결과: {implementation_end_date}")
        return implementation_end_date
    except Exception as e:
        print(f"시행 마감 날짜 라벨링 실패: {e}")
        return "시행 마감 날짜 없음"




# Google Drive에서 파일 목록 가져오기
def get_txt_files_from_drive(folder_id):
    query = f"'{folder_id}' in parents and mimeType='text/plain'"
    results = drive_service.files().list(q=query, fields="files(id, name)").execute()
    return results.get('files', [])

# Google Drive에서 파일 내용 가져오기
def download_file(file_id):
    request = drive_service.files().get_media(fileId=file_id)
    file_content = request.execute()
    return file_content.decode('utf-8')

# Google Drive의 텍스트 파일 처리
files = get_txt_files_from_drive(shared_folder_id)
for idx, file in enumerate(files):
    file_content = download_file(file['id']).strip()
    
    if not file_content:
        print(f"파일 '{file['name']}'이 비어 있어 처리되지 않았습니다.")
        continue

    # 주제 라벨링
    label = label_topic(file_content)
    if not label:
        print(f"파일 '{file['name']}'의 주제 라벨 생성 실패로 저장되지 않았습니다.")
        continue
    print(f"파일 '{file['name']}'에 대한 주제 라벨: {label}")

    # 지역 라벨링
    region = label_region(file_content)
    if not region:
        print(f"파일 '{file['name']}'의 지역 라벨 생성 실패로 저장되지 않았습니다.")
        continue
    print(f"파일 '{file['name']}'에 대한 지역 라벨: {region}")

    # 모집 및 시행 날짜 라벨링
    recruit_start_date = label_recruit_start_date(file_content).strip().strip("'")
    recruit_end_date = label_recruit_end_date(file_content).strip().strip("'")
    implementation_start_date = label_implementation_start_date(file_content).strip().strip("'")
    implementation_end_date = label_implementation_end_date(file_content).strip().strip("'")
    
    print(f"파일 '{file['name']}'의 라벨링 결과:")
    print(f"  - 모집 시작: {recruit_start_date}")
    print(f"  - 모집 마감: {recruit_end_date}")
    print(f"  - 시행 시작: {implementation_start_date}")
    print(f"  - 시행 마감: {implementation_end_date}")


    # 임베딩 생성
    try:
        embedding = model.encode([file_content])[0]
    except Exception as e:
        print(f"파일 '{file['name']}'의 임베딩 생성 실패: {e}")
        continue

    # 데이터 저장
    try:
        collection.add(
            documents=[file_content],
            embeddings=[embedding],
            metadatas=[
                {
                    "label": label,
                    "region": region,
                    "모집 시작": recruit_start_date,
                    "모집 마감": recruit_end_date,
                    "시행 시작": implementation_start_date,
                    "시행 마감": implementation_end_date,
                }
            ],
            ids=[f"doc_{idx+1}"]
        )
        print(f"파일 '{file['name']}'의 데이터를 성공적으로 저장했습니다.")
    except ValueError as e:
        print(f"파일 '{file['name']}'의 데이터 저장 실패: {e}")

In [ ]:
# chromaDB csv파일로 내보내는 코드
import csv

def export_chromadb_to_csv(collection, file_name="chromadb_data.csv"):
    try:
        # ChromaDB 컬렉션에서 모든 데이터 가져오기
        results = collection.get(include=["documents", "metadatas", "embeddings"])
        
        # 데이터의 ID는 별도로 관리되므로 수동으로 추가
        ids = collection.get()["ids"]
        
        with open(file_name, mode="w", encoding="utf-8", newline="") as csvfile:
            writer = csv.writer(csvfile)
            # CSV 헤더 작성
            writer.writerow(["ID", "Document", "Metadata", "Embedding"])
            
            # 데이터 작성
            for doc_id, document, metadata, embedding in zip(
                ids, results["documents"], results["metadatas"], results["embeddings"]
            ):
                writer.writerow([doc_id, document, metadata, embedding])
        
        print(f"ChromaDB 데이터가 '{file_name}'로 저장되었습니다.")
    except Exception as e:
        print(f"CSV로 내보내는 데 실패했습니다: {e}")

# CSV로 내보내기 실행
export_chromadb_to_csv(collection)
